In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import numpy as np
from datetime import datetime

In [2]:
dataset, info = tfds.load('cifar10',as_supervised=True,
    split = [tfds.Split.TRAIN.subsplit(tfds.percent[:80]),
        tfds.Split.TRAIN.subsplit(tfds.percent[80:90]),
        tfds.Split.TRAIN.subsplit(tfds.percent[90:]),],
    with_info=True
)

In [3]:
dataset

[<_OptionsDataset shapes: ((32, 32, 3), ()), types: (tf.uint8, tf.int64)>,
 <_OptionsDataset shapes: ((32, 32, 3), ()), types: (tf.uint8, tf.int64)>,
 <_OptionsDataset shapes: ((32, 32, 3), ()), types: (tf.uint8, tf.int64)>]

In [4]:
info

tfds.core.DatasetInfo(
    name='cifar10',
    version=1.0.2,
    description='The CIFAR-10 dataset consists of 60000 32x32 colour images in 10 classes, with 6000 images per class. There are 50000 training images and 10000 test images.',
    homepage='https://www.cs.toronto.edu/~kriz/cifar.html',
    features=FeaturesDict({
        'image': Image(shape=(32, 32, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    total_num_examples=60000,
    splits={
        'test': 10000,
        'train': 50000,
    },
    supervised_keys=('image', 'label'),
    citation="""@TECHREPORT{Krizhevsky09learningmultiple,
        author = {Alex Krizhevsky},
        title = {Learning multiple layers of features from tiny images},
        institution = {},
        year = {2009}
    }""",
    redistribution_info=,
)

In [5]:
train_dataset = dataset[0]
validation_dataset = dataset[1]
test_dataset = dataset[2]

In [6]:
def preprocess(image, label):
    image = tf.dtypes.cast(image, tf.float32)
    label = tf.dtypes.cast(label, tf.float32)
    image = tf.image.per_image_standardization(image)
    return image, label

In [7]:
train_dataset = train_dataset.map(preprocess)
validation_dataset = validation_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


Instructions for updating:
Deprecated in favor of operator or tf.math.divide.


In [8]:
class LeNet(tf.keras.Model):
    def __init__(self):
        super(LeNet, self).__init__()
        initializer = tf.initializers.GlorotUniform(seed=123)
        # Conv1
        self.wc1 = tf.Variable(initializer([3, 3, 3, 10]), trainable=True, name='wc1')
        
        # Conv2
        self.wc2 = tf.Variable(initializer([3, 3, 10, 20]), trainable=True, name='wc2')
        
        # Conv3
        self.wc3 = tf.Variable(initializer([3, 3, 20, 40]), trainable=True, name='wc3')
        
        # Flatten
        
        # Dense
        self.wd3 = tf.Variable(initializer([640, 280]), trainable=True)
        self.wd4 = tf.Variable(initializer([280, 80]), trainable=True)        
        self.wd5 = tf.Variable(initializer([80, 10]), trainable=True)
        
        self.bc1 = tf.Variable(tf.zeros([10]), dtype=tf.float32, trainable=True)
        self.bc2 = tf.Variable(tf.zeros([20]), dtype=tf.float32, trainable=True)
        self.bc3 = tf.Variable(tf.zeros([40]), dtype=tf.float32, trainable=True)
        
        self.bd3 = tf.Variable(tf.zeros([280]), dtype=tf.float32, trainable=True)
        self.bd4 = tf.Variable(tf.zeros([80]), dtype=tf.float32, trainable=True)        
        self.bd5 = tf.Variable(tf.zeros([10]), dtype=tf.float32, trainable=True)   
    
    def call(self, x):
        # X = NHWC 
        # Conv1 + maxpool 2
        x = tf.nn.conv2d(x, self.wc1, strides=[1, 1, 1, 1], padding="SAME")
        x = tf.nn.bias_add(x, self.bc1)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
        
        # Conv2 + maxpool 2
        x = tf.nn.conv2d(x, self.wc2, strides=[1, 1, 1, 1], padding="SAME")
        x = tf.nn.bias_add(x, self.bc2)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
        
        # Conv3 + maxpool 3
        x = tf.nn.conv2d(x, self.wc3, strides=[1, 1, 1, 1], padding="SAME")
        x = tf.nn.bias_add(x, self.bc3)
        x = tf.nn.relu(x)
        x = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")
        
        # Flattten out
        # N X Number of Nodes
        # Flatten()
        x = tf.reshape(x, (tf.shape(x)[0], -1))
        
        # Dense1
        x = tf.matmul(x, self.wd3)
        x = tf.nn.bias_add(x, self.bd3)
        x = tf.nn.relu(x)

        
        # Dense2
        x = tf.matmul(x, self.wd4)
        x = tf.nn.bias_add(x, self.bd4)
        x = tf.nn.relu(x)
        
        
        # Dense3
        x = tf.matmul(x, self.wd5)
        x = tf.nn.bias_add(x, self.bd5)
#         x = tf.nn.sigmoid(x)
        
        return x

In [10]:
model = LeNet()

In [11]:
ce_loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [12]:
LEARNING_RATE = 0.001

In [13]:
optimizer = tf.optimizers.Adam(learning_rate=LEARNING_RATE)

In [14]:
def train_step(model, inputs, labels, loss_fn, optimzer):
    with tf.GradientTape() as t:
        y_predicted = model(inputs, training=True)
        current_loss = loss_fn(labels, y_predicted)

    gradients = t.gradient(current_loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return current_loss

In [15]:
def valid_step(model, inputs, labels, loss_fn):
    y_predicted = model(inputs, training=False)
    current_loss = loss_fn(labels, y_predicted)
    return current_loss

In [16]:
EPOCHS = 20
BATCH_SIZE = 32

In [17]:
train_dataset = train_dataset.batch(BATCH_SIZE)
validation_dataset = validation_dataset.batch(BATCH_SIZE)

In [18]:
sample = np.random.randn(1, 32, 32, 3)

In [19]:
filters = tf.Variable(np.random.randn(3, 3, 3, 10))

In [20]:
len(model.trainable_variables)

12

In [21]:
losses = tf.keras.metrics.Mean(name='loss')
val_losses = tf.keras.metrics.Mean(name='val_loss')

In [22]:
current_time = datetime.now().strftime("%Y%m%d-%H%M%S")
train_log_dir = f'./trainLog/{current_time}/logs'
file_writer = tf.summary.create_file_writer(train_log_dir)

In [23]:
checkpoint_dir = f'./ckpt/'

In [24]:
ckpt = tf.train.Checkpoint(step = tf.Variable(1), optimizer=optimizer, net=model)
manager = tf.train.CheckpointManager(ckpt, checkpoint_dir, max_to_keep=3)


In [25]:
def check_for_checkpoint(manager):
    ckpt.restore(manager.latest_checkpoint)
    if manager.latest_checkpoint:
        print(f"restored from {manager.latest_checkpoint}")
    else:
        print("Initializing from scratch")

In [26]:
check_for_checkpoint(manager)

Initializing from scratch


In [27]:
for epoch in range(EPOCHS):
    ckpt.step.assign_add(1)

    print(f'epoch: {epoch}')
    losses.reset_states()
    val_losses.reset_states()
    for x_batch, y_batch in train_dataset:
        loss = train_step(model, x_batch, y_batch, ce_loss, optimizer)
        losses(loss)
#         step += 1
    
    save_path = manager.save()
    print("Saved checkpoint for step {}: {}".format(int(ckpt.step), save_path))

    with file_writer.as_default():
        tf.summary.scalar('loss', losses.result(), step=epoch)
        tf.summary.image('Input images', x_batch, step=epoch)

    print(losses.result())
        
    for x_batch, y_batch in validation_dataset:
        val_loss = valid_step(model, x_batch, y_batch, ce_loss)
        val_losses(val_loss)
    
    with file_writer.as_default():
        tf.summary.scalar('val_loss', val_losses.result(), step=epoch)


epoch: 0
Saved checkpoint for step 2: ./ckpt/ckpt-1
tf.Tensor(1.3936287, shape=(), dtype=float32)
epoch: 1
Saved checkpoint for step 3: ./ckpt/ckpt-2
tf.Tensor(1.0418618, shape=(), dtype=float32)
epoch: 2
Saved checkpoint for step 4: ./ckpt/ckpt-3
tf.Tensor(0.87765324, shape=(), dtype=float32)
epoch: 3
Saved checkpoint for step 5: ./ckpt/ckpt-4
tf.Tensor(0.7588274, shape=(), dtype=float32)
epoch: 4
Saved checkpoint for step 6: ./ckpt/ckpt-5
tf.Tensor(0.6598256, shape=(), dtype=float32)
epoch: 5
Saved checkpoint for step 7: ./ckpt/ckpt-6
tf.Tensor(0.58142835, shape=(), dtype=float32)
epoch: 6
Saved checkpoint for step 8: ./ckpt/ckpt-7
tf.Tensor(0.5100259, shape=(), dtype=float32)
epoch: 7
Saved checkpoint for step 9: ./ckpt/ckpt-8
tf.Tensor(0.44460648, shape=(), dtype=float32)
epoch: 8
Saved checkpoint for step 10: ./ckpt/ckpt-9
tf.Tensor(0.38922107, shape=(), dtype=float32)
epoch: 9
Saved checkpoint for step 11: ./ckpt/ckpt-10
tf.Tensor(0.33566892, shape=(), dtype=float32)
epoch: 10
Sa

In [34]:
# tf.train.list_variables(manager.latest_checkpoint)

In [29]:
def predict(inputs):
    predicted = model(inputs)
    return tf.nn.softmax(predicted)

In [35]:
# for sample, label in test_dataset.batch(2).take(1):
#     predictions = predict(sample)
#     print(tf.argmax(predictions, axis=1), label)

In [31]:
import os

In [32]:
model_dir = f'./models/'
weights_path = os.path.join(model_dir, 'weights.h5')
model.save_weights(weights_path)